# Enviando ruído para CNN

## Definindo a arquitetura e funções de treino e teste

In [ ]:
#Códigos e Rede adaptados de https://github.com/pytorch/examples/blob/main/mnist/main.py
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.dropout1 = nn.Dropout(0.25)
    self.dropout2 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output


def train(model, device, train_loader, optimizer, epoch, criterion):
  model.train()
  running_loss = 0.0
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print(f'Epoch [{epoch}], Loss: {running_loss/len(train_loader):.4f}')

def test(model, device, test_loader, criterion):
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)

    with torch.no_grad():
      output = model(data)

    test_loss += criterion(output, target, reduction='sum').item()
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

## Define dataset personalizado

In [ ]:
class CustomImageDataset(Dataset):
  def __init__(self, image_dir, transform=None):
    self.image_dir = image_dir
    self.transform = transform
    self.image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, img)) and img.lower().endswith('.bmp')]

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx):
    img_path = self.image_paths[idx]
    image = Image.open(img_path)
    if self.transform:
        image = self.transform(image)
    return image, img_path

## Preparando dataloaders com o dataset do MNIST

In [ ]:
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1024}
cuda_kwargs = {'num_workers': 1, 'shuffle': True}
train_kwargs.update(cuda_kwargs)
test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

print("train loader lenght:", len(train_loader))
print("test loader lenght:", len(test_loader))

## Treino e Teste da Rede

In [ ]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
criterion = F.nll_loss

epochs = 4
for epoch in range(1, epochs + 1):
  train(model, device, train_loader, optimizer, epoch, criterion)
  scheduler.step()

test(model, device, test_loader, criterion)

## Inferencia da rede com imagens aleatórias

In [ ]:
#Pegando algumas imagens aleatórias de teste e enviando para a rede
random_indices = random.sample(range(len(dataset2)), 10)
images, labels = zip(*[dataset2[i] for i in random_indices])

fig, axes = plt.subplots(1, 10, figsize=(15, 1.5))
for img, ax in zip(images, axes):
  ax.imshow(img.squeeze(), cmap='gray')
  ax.axis('off')
plt.show()

images_tensor = torch.stack(images).to(device)

model.eval()
with torch.no_grad():
  output = model(images_tensor)

_, predicted = torch.max(output, 1)
print(predicted)

## Classificando imagens do diretório

In [ ]:
image_dir = "/content/imgs/"

custom_dataset = CustomImageDataset(image_dir, transform=transform)
data_loader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

# Classificar as imagens do diretório
for images, img_paths in data_loader:
  images = images.to(device)

  with torch.no_grad():
    output = model(images)

  probabilities = torch.softmax(output, dim=1)
  _, predicted = torch.max(output, 1)

  fig, axes = plt.subplots(1, len(images), figsize=(15, 2))
  for i, (img_path, pred, prob, image) in enumerate(zip(img_paths, predicted, probabilities, images)):
    ax = axes[i] if len(images) > 1 else axes
    img = Image.open(img_path)
    ax.imshow(img, cmap='gray')
    title = f"Pred: {pred.item()}\nProb: {prob[pred].item():.2f}"
    ax.set_title(title)
    ax.axis('off')
  plt.show()

## Classificando imagens aleatórias (ruído)

In [ ]:
def generate_random_image(size=(28, 28, 1)):
    return np.random.rand(*size).astype(np.float32)

num_images = 10
images = []

for i in range(num_images):
  noise_image = generate_random_image()
  images.append(noise_image)

fig, axes = plt.subplots(1, num_images, figsize=(20, 2))
for i, ax in enumerate(axes):
    ax.imshow(images[i])
    ax.axis('off')
plt.show()

noise_images_tensor = torch.stack([torch.from_numpy(img).permute(2, 0, 1) for img in images])
noise_images_tensor = noise_images_tensor.to(device)

with torch.no_grad():
  output = model(noise_images_tensor)

probabilities = F.softmax(output, dim=1)
_, predicted = torch.max(probabilities, 1)

for i, (pred, prob) in enumerate(zip(predicted, probabilities)):
  print(f"Pred: {pred.item()} Prob: {prob[pred].item():.2f}")

# Refazendo os testes para MLP convencional

## Define arquitetura da MLP

In [ ]:
class MLP(nn.Module):
  def __init__(self, input_size, hidden_sizes, output_size):
    super(MLP, self).__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(input_size, hidden_sizes[0])
    self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
    self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
    self.fc4 = nn.Linear(hidden_sizes[2], output_size)
    self.relu = nn.ReLU()
    self.dropout_input = nn.Dropout(0.1)
    self.dropout_hidden = nn.Dropout(0.2)

  def forward(self, x):
    x = self.flatten(x)
    x = self.dropout_input(x)
    x = self.relu(self.fc1(x))
    x = self.dropout_hidden(x)
    x = self.relu(self.fc2(x))
    x = self.dropout_hidden(x)
    x = self.relu(self.fc3(x))
    x = self.dropout_hidden(x)
    x = self.fc4(x)
    return x

## Treino e Teste da MLP

In [ ]:
input_size = 28 * 28
hidden_sizes = [512, 256, 128]
output_size = 10

model = MLP(input_size, hidden_sizes, output_size)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 4
for epoch in range(1, epochs + 1):
  train(model, device, train_loader, optimizer, epoch, criterion)

model.eval()

correct = 0
total = 0
for images, labels in test_loader:
  images, labels = images.to(device), labels.to(device)

  with torch.no_grad():
    outputs = model(images)

  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

## Usando imagens aleatórias para o teste

In [ ]:
#Pegando algumas imagens aleatórias de teste e enviando para a rede
random_indices = random.sample(range(len(dataset2)), 10)
images, labels = zip(*[dataset2[i] for i in random_indices])

fig, axes = plt.subplots(1, 10, figsize=(15, 1.5))
for img, ax in zip(images, axes):
  ax.imshow(img.squeeze(), cmap='gray')
  ax.axis('off')
plt.show()

images_tensor = torch.stack(images).to(device)

model.eval()
with torch.no_grad():
    output = model(images_tensor)

_, predicted = torch.max(output, 1)
print(predicted)

## Classificando imagens do diretório

In [ ]:
# Classificar as imagens do diretório
for images, img_paths in data_loader:
    images = images.to(device)
    with torch.no_grad():
      output = model(images)

    probabilities = torch.softmax(output, dim=1)
    _, predicted = torch.max(output, 1)

    fig, axes = plt.subplots(1, len(images), figsize=(15, 2))
    for i, (img_path, pred, prob, image) in enumerate(zip(img_paths, predicted, probabilities, images)):
      ax = axes[i] if len(images) > 1 else axes
      img = Image.open(img_path)
      ax.imshow(img, cmap='gray')
      title = f"Pred: {pred.item()}\nProb: {prob[pred].item():.2f}"
      ax.set_title(title)
      ax.axis('off')
    plt.show()

## Classificando imagens aleatórias (ruído) com MLP

In [ ]:
images = []

for i in range(num_images):
    noise_image = generate_random_image()
    images.append(noise_image)

fig, axes = plt.subplots(1, num_images, figsize=(20, 2))
for i, ax in enumerate(axes):
    ax.imshow(images[i])
    ax.axis('off')
plt.show()

noise_images_tensor = torch.stack([torch.from_numpy(img).permute(2, 0, 1) for img in images])
noise_images_tensor = noise_images_tensor.to(device)

with torch.no_grad():
    output = model(noise_images_tensor)

probabilities = F.softmax(output, dim=1)
_, predicted = torch.max(probabilities, 1)

for i, (pred, prob) in enumerate(zip(predicted, probabilities)):
  print(f"Pred: {pred.item()} Prob: {prob[pred].item():.2f}")